In [1]:
from neo4j import GraphDatabase
import re
import nltk

from typing import List

In [2]:
driver = GraphDatabase.driver("bolt://172.30.48.1:7687/",
                              auth=("neo4j", "1234"))
driver.verify_connectivity()


In [3]:
lmn = nltk.stem.WordNetLemmatizer()

def addMetaWord(tx, first: str, connectedWords: List[str] = None):
    if connectedWords is None:
        tx.run("MERGE (a:Source {name: $source}) "
               "MERGE (b:Word {word: $first}) "
               "SET b :Meta "
               "MERGE (a)<-[:Src]-(b) ", 
               source="VUA", first=lmn.lemmatize(first))
    else:
        for sec in connectedWords:
            tx.run("MERGE (a:Source {name: $source}) "
                   "MERGE (b:Word {word: $first}) "
                   "SET b :Meta "
                   "MERGE (c:Word {word: $second}) "
                   "SET c :Meta "
                   "MERGE (a)-[:Src]->(b) "
                   "MERGE (b)-[:META]-(c) ", 
                   source="VUA", first=lmn.lemmatize(first), second=lmn.lemmatize(sec))

In [4]:
delim = ','

with open("./DeepMet/data/VUA/VUA_train_features2.csv", "r") as f:
    labelIdx = {}
    for i, lbl in enumerate(f.readline().split(delim)):
        if lbl in ['id']:
            labelIdx[lbl.strip('\n')] = i
        else:
            labelIdx[lbl.strip('\n')] = i - 1
    
    count = 0
    sections = {}

    #Reading in the data
    while ((data := f.readline())):
        data = re.sub('"[^"]+",', '', data).split(delim)

        if data[labelIdx['label']] == '1':
            if data[labelIdx['local']] not in sections:
                sections[data[labelIdx['local']]] = []
            if data[labelIdx['word']] not in sections[data[labelIdx['local']]]:
                sections[data[labelIdx['local']]].append(data[labelIdx['word']])
        count += 1
    
    #Adding it to the database
    with driver.session(database='neo4j') as session:
        for local, words in sections.items():
            if len(words) == 1:
                session.execute_write(addMetaWord, words[0])
            else:
                session.execute_write(addMetaWord, words[0], words[1:])
